#### 사전준비

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive') 

In [ ]:
# cd drive/MyDrive/Colab Notebooks

In [ ]:
!pip install konlpy
!pip install tensorflow
!pip install pandas

In [ ]:
!pip install matplotlib

In [ ]:
!pip install sklearn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from konlpy.tag import Okt
from tensorflow.keras import layers, Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv("test.csv")

In [ ]:
x = train['사람문장1']
y = train['감정_대분류']

In [ ]:
X = test['사람문장1']
Y = test['감정_대분류']

In [ ]:
stop_words = '께 을 를 에게 은 는 이 가 . 게 로 에'.split()

#### okt 형태소 분류

In [ ]:
okt = Okt()

token_sentence = []
for sentence in x:
  tokens = []
  for word in okt.pos(sentence,stem = True):
    if word[0] not in stop_words:
      tokens.append(word[0])
  token_sentence.append(tokens)

In [ ]:
x_train = pd.DataFrame(x)
x_train['token_sentence'] = token_sentence
x_train

In [ ]:
okt = Okt()

token_sentence = []
for sentence in X:
  tokens = []
  for word in okt.pos(sentence,stem = True):
    if word[0] not in stop_words:
      tokens.append(word[0])
  token_sentence.append(tokens)

In [ ]:
x_test = pd.DataFrame(X)
x_test['token_sentence'] = token_sentence
x_test

#### 벡터화

In [ ]:
# tokenizer=Tokenizer()
# tokenizer.fit_on_texts(x_train["token_sentence"])

In [ ]:
# print(len(tokenizer.word_index))

In [ ]:
# x_train=tokenizer.texts_to_sequences(x_train['token_sentence'])
# x_train

In [ ]:
# num_token_per_sentence = [ len(sentence) for sentence in x_train ]
# print(max(num_token_per_sentence))

# plt.hist(num_token_per_sentence, bins=20)
# plt.show()

In [ ]:
# x_train_pad=pad_sequences(x_train,maxlen=38,padding='post')
# x_train_pad

In [ ]:
y_train=y.replace({'기쁨':0,'당황':1,'분노':2,'불안':3,'상처':4,'슬픔':5})
y_train

In [ ]:
# tokenizer=Tokenizer()
# tokenizer.fit_on_texts(x_test["token_sentence"])
# x_test=tokenizer.texts_to_sequences(x_test['token_sentence'])

In [ ]:
# num_token_per_sentence = [ len(sentence) for sentence in x_test ]
# print(max(num_token_per_sentence))

# plt.hist(num_token_per_sentence, bins=20)
# plt.show()

In [ ]:
# x_test_pad=pad_sequences(x_test,maxlen=38,padding='post')
# x_test_pad

In [ ]:
y_test=Y.replace({'기쁨':0,'당황':1,'분노':2,'불안':3,'상처':4,'슬픔':5})
y_test

#### tf-idf

In [ ]:
x_train

In [ ]:
tfidv = TfidfVectorizer().fit(map(' '.join,x_train['token_sentence']))

In [ ]:
tfidv.fit_transform((map(' '.join,x_train["token_sentence"])))

x_train = pd.DataFrame(tfidv.transform(x).toarray())

#### ML

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, SimpleRNN, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

In [ ]:
# len(tokenizer.word_index)

In [ ]:
sequence_length = 9238 # max(num_token_per_sentence)
vocabulary_size = 40879
embedding_dim = 256

In [ ]:
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_dim, input_length=sequence_length))
model.add(Dropout(0.25))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(6, activation='softmax'))

print(model.summary())

In [ ]:
# model = Sequential()
# model.add(Embedding(vocabulary_size, embedding_dim, input_length=sequence_length))
# model.add(Dropout(0.25))
# model.add(SimpleRNN(64,return_sequences=True))
# model.add(SimpleRNN(128,return_sequences=True))
# model.add(SimpleRNN(256))
# model.add(Dense(256, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(6, activation='softmax'))

# print(model.summary())

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=2048)

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label='Training Accuracy')
plt.plot(epochs_range, val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test_pad, y_test)

print('test set accuracy: ', test_accuracy)

In [ ]:
model.save("saved_model"+test_accuracy)